In [1]:
from pathlib import Path

import pandas as pd

from rtnls_fundusprep.preprocessor import parallel_preprocess

## Preprocessing

This code will preprocess the images and write .png files with the square fundus image and the contrast enhanced version

This step is not strictly necessary, but it is useful if you want to run the preprocessing step separately before model inference


Create a list of files to be preprocessed:

In [2]:
ds_path = Path("../data/LES-AV")
files = list((ds_path / "original").glob("*"))

Images with .dcm extension will be read as dicom and the pixel_array will be read as RGB. All other images will be read using PIL's Image.open

In [3]:
bounds = parallel_preprocess(
    files,  # List of image files
    rgb_path=ds_path / "rgb",  # Output path for RGB images
    ce_path=ds_path / "ce",  # Output path for Contrast Enhanced images
    n_jobs=4,  # number of preprocessing workers
)
df_bounds = pd.DataFrame(bounds).set_index("id")

0it [00:00, ?it/s][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
8it [00:01,  4.39it/s][Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.1s
16it [00:02,  7.44it/s][Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.4s
22it [00:02,  7.90it/s]
[Parallel(n_jobs=4)]: Done  18 out of  22 | elapsed:    3.0s remaining:    0.6s
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    3.2s finished


The preprocessor will produce RGB and contrast-enhanced preprocessed images cropped to a square and return a dataframe with the image bounds that can be used to reconstruct the original image. Output files will be named the same as input images, but with .png extension. Be careful with providing multiple inputs with the same filename without extension as this will result in over-written images. Any exceptions during pre-processing will not stop execution but will print error. Images that failed pre-processing for any reason will be marked with `success=False` in the df_bounds dataframe.

In [4]:
df_bounds.to_csv(ds_path / "meta.csv")